# ラクマの分類タスク


## インポート


In [1]:
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re
import os
from pathlib import Path

BASE_DIR = Path().resolve().parent
os.chdir(BASE_DIR)


## データチェック


In [2]:
df = pd.read_csv("data/rakuma.csv")


In [3]:
ctg_df = df.category.value_counts().reset_index()  # reset_indexでデータフレーム形式になる
ctg_df = ctg_df.rename(columns={"index": "category_name", "category": "num"})


In [4]:
df_ctg_1000 = ctg_df[ctg_df.num > 1000]


In [5]:
df_ctg_1000.num.sum()


63006

In [6]:
data = df.merge(
    df_ctg_1000[["category_name"]], left_on="category", right_on="category_name"
)


In [7]:
data.category.nunique()


22

## テキストの前処理


In [8]:
emojis = "".join(emoji.EMOJI_DATA.keys())


In [9]:
puncs = string.punctuation + "◆▼★②●☆■★【】『』「」、♪"


In [10]:
def han2zen(txt):
    txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
    return jctconv.z2h(txt, kana=False, digit=True, ascii=True)


def remove_signs(txt):
    rm_signs = emojis + puncs
    for s in rm_signs:
        txt = txt.replace(s, " ")
    return txt


def clean_txt(txt):
    txt = han2zen(txt)
    txt = remove_signs(txt)
    txt_list = txt.upper().split()
    txt_list = [
        x for x in txt_list if len(x) > 1 and re.search(r"[亜-熙ぁ-んァ-ヶa-zA-Z]", x)
    ]
    return " ".join(list(Counter(txt_list)))


In [11]:
t = " ".join(["bike", "bike", "handle", "saddle", "2020", "#"])
print("t: ", t)
print(clean_txt(t))


t:  bike bike handle saddle 2020 #
BIKE HANDLE SADDLE


In [12]:
data.title.head()


0       テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1       送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                      ☆ウルトラマン☆レーザーディスク ジャンク？
3         MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4    LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
Name: title, dtype: object

In [13]:
data["clean_title"]=data.title.apply(clean_txt)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB


In [15]:
category2idx = {c: idx for idx, c in enumerate(data.category_name.unique())}
idx2category = {idx: c for idx, c in enumerate(data.category_name.unique())}


In [16]:
X=data.clean_title
y=data.category_name.apply(lambda x:category2idx[x])


## データを学習させる

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42,shuffle=True
)

In [18]:
X_train

12762      MINT 新品未使用 メダル大集合 ドラえもん ポケモン カーズ
11121                 すみっコぐらし ブラシ付ミラー 新品未使用
47140        ADIDAS PHARRELL HUMAN RACE NMD
62381                          PS ウイニングイレブン
22213                       ジョジョと奇妙な冒険 シーザー
                        ...                
62570    コールオブデューティアドバンスド・ウォーフェア CALLOFDUTY
38158          新品 鴻池剛 ニャアアアン 猫♡ぽんた♡スウェット♡半袖
860               廃盤LP 沢田研二 AUX FEMMES 女たちよ
15795         MINT レア キリ番 マイメロディ バースデーマスコット
56422               掛軸 小林雄山 四季花図 絹本 共箱付 掛け軸
Name: clean_title, Length: 44104, dtype: object

In [19]:
pipeline = Pipeline(
    [
        ("bow", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("classifier", RandomForestClassifier()),
    ]
)

In [20]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 44104 entries, 12762 to 56422
Series name: category_name
Non-Null Count  Dtype
--------------  -----
44104 non-null  int64
dtypes: int64(1)
memory usage: 689.1 KB


In [21]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [22]:
pred = pipeline.predict(X_test)

In [23]:
# RandomForestClassifier
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77       370
           1       0.90      0.93      0.91      1077
           2       0.91      0.80      0.85       377
           3       0.89      0.87      0.88       632
           4       0.77      0.83      0.80      3252
           5       0.73      0.38      0.50       355
           6       0.88      0.82      0.85      1936
           7       0.85      0.83      0.84       934
           8       0.92      0.90      0.91       407
           9       0.93      0.82      0.87       355
          10       0.97      0.85      0.90       782
          11       0.83      0.57      0.68       440
          12       0.89      0.77      0.82       398
          13       0.94      0.82      0.87       371
          14       0.99      0.95      0.97       481
          15       0.95      0.91      0.93       760
          16       0.98      0.81      0.89       356
          17       0.98    

In [26]:
target = ["ビームス　ナイキ　REACT PRESTO DHARMA"]
prediction = pipeline.predict(target)
prediction

array([17])

In [25]:
idx2category

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

In [28]:
import pickle

with open("./model/rdmf.pickle", mode="wb") as f:
    pickle.dump(pipeline, f)